<a href="https://colab.research.google.com/github/4rldur0/whyfi/blob/xeoyeon/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 초기 설정

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/RAG

/content/drive/MyDrive/Colab Notebooks/RAG


In [3]:
!pip install transformers chromadb langchain langchain_community langchain-chroma

# 2. data 불러오기

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/RAG/dataset/cleaned_word_dict.csv'
data = pd.read_csv(file_path)

## pdf 가져와서 chunk하기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_name = # 읽어오려는 파일 경로

loader = PyPDFLoader(file_name)
pages = loader.load()
text = ""
for page in pages:
    sub = page.page_content
    text += sub

In [ ]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# token size 기준으로 contents split
tokenizer = AutoTokenizer.from_pretrained(ENCODER)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer,
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            separator="\n" # default: "\n\n"
        )


documents=[] # split 한 문서들을 담기 위한 array

split_conts = text_splitter.split_text(text)
for chunk_idx, split_cont in enumerate(split_conts):
    documents.append(Document(
        page_content=split_cont,
        metadata={
            "file_name": file_name,
        },
        id=chunk_idx,
    ))
    idx+=1

vectorstore.add_documents(documents)

## website 문서 로딩하기

In [ ]:
# Load documents from web
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader([
    "링크 삽입"
    ]
)

data = web_loader.load()

In [ ]:
##Split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

all_splits = text_splitter.split_documents(data)

all_splits[0]
출처: https://rfriend.tistory.com/832 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]

# 3. chromaDB에 데이터 저장하기

In [4]:
!pip install langchain-huggingface

  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
Using cached langchain_huggingface-0.1.2-py3-none-any.whl (21 kB)


In [5]:
#기존 임베딩 결과만 가져오기
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정 (1000개로 확장)
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# 기존 컬렉션 삭제
vectorstore._client.delete_collection("Chroma_Collection")

# 새로운 컬렉션 생성
vectorstore = Chroma(
    collection_name="Chroma_Collection",
    embedding_function=embedding_func,
    persist_directory="/content/drive/MyDrive/Colab Notebooks/RAG",
    collection_metadata={"max_size": 1000}
)


In [ ]:
from langchain.docstore.document import Document # 데이터의 각 row를 document 객체로 변환하여 저장하기 위함

# vectorDB에 data 추가하는 함수
def add_data_to_vectorstore(data, vectorstore):
  for index, row in data.iterrows(): # row라는 변수에 각 행을 반복적으로 가져옴.
    text = row.get("Content","")
    metadata = row.to_dict() #행 전체를 딕셔너리 형태로 변환
    metadata["source"] = metadata.get("source", f"row_{index}")   # source 필드 추가 (기본값으로 행 번호를 사용하거나 특정 열에서 가져오기)
    document = Document(page_content=text, metadata=metadata) #cocument 객체를 생성
    vectorstore.add_texts([document.page_content],[document.metadata]) # db에 데이터 추가

add_data_to_vectorstore(data, vectorstore)

# 4. Retriever 및 프롬프트 설정

In [6]:
# Define a retriever to search in the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) #검색 시 가장 관련성 높은 3개의 문서를 반환하라는 뜻

In [7]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# system_template="""
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

# You MUST answer in Korean.

# Question: {question}
# Context: {context}
# Answer:
# """
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어나 개념을 최대 3개까지 추천하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

# 5. gemini api 설정 및 chain 구현하기

In [8]:
!pip install langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.74.0
    Uninstalling google-cloud-aiplatform-1.74.0:
      Successfully uninstalled google-cloud-aiplatform-1.74.0


In [9]:
PROJECT_ID = "your_project_id"
REGION = "your_region"

from google.colab import auth
auth.authenticate_user()

In [38]:
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [11]:
# Chain 사용 예시
question = "가동률이란?"
result = chain({"question": question})

print("Answer:", result["answer"])
print("Sources:", result["source_documents"])

<ipython-input-11-22bc5efff800>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": question})


Answer: ## 가동률이란?

가동률은 **생산설비가 얼마나 활발하게 이용되고 있는지를 나타내는 지표**입니다. 쉽게 말해, **생산 능력 대비 실제 생산량의 비율**을 의미합니다. 

예를 들어, 한 공장의 생산 능력이 100개의 제품을 생산할 수 있다면, 실제로 70개의 제품을 생산했다면 가동률은 70%가 됩니다. 

가동률은 경기 상황을 파악하는 데 중요한 지표입니다. 경기가 좋으면 기업들은 더 많은 제품을 생산하기 위해 가동률을 높이고, 반대로 경기가 나쁘면 가동률을 낮춥니다. 

## 가동률을 이해하는 데 도움이 되는 실제 사례

* **제조업**: 한 제조업체가 생산 능력의 80%를 가동하고 있다면, 이는 경기가 비교적 좋고, 앞으로도 더 많은 제품을 생산할 계획이 있음을 의미할 수 있습니다.
* **서비스업**: 한 레스토랑이 주말에는 100%의 가동률을 보이지만, 평일에는 50%의 가동률을 보인다면, 이는 주말에 더 많은 손님이 찾는다는 것을 의미합니다.

## 가동률과 관련된 다른 용어

* **생산 능력**: 사업체가 정상적인 조업 환경에서 생산할 수 있는 최대 생산량을 의미합니다.
* **제조업 평균 가동률**: 제조업 부문의 생산설비 이용도를 보여주는 지표입니다.
* **제조업 평균 가동률 갭**: 한 시점에서의 제조업 평균 가동률이 잠재 수준에서 얼마나 벗어나 있는지를 나타내는 지표입니다.

## 궁금한 점이 있으시면 언제든지 질문해 주세요!
Sources: [Document(id='6dbbccaf-d26a-4200-9afa-84207b2822ca', metadata={'Content': '가동률은 사업체의 생산설비 이용도를 나타내는 지표이다. 통상 가동률은 경기상황이 좋지 않아 제품의 판매량이 감소할 경우 사업체가 생산량을 축소조정하고 불필요한 장비의 가동도 줄여나가면서 하락하게 된다. 반대로 경제상황이 호전되면 가동률은 상승한다. 구체적으로 가동률은 생산능력 대비 실제 생산실적의 비율을 말한다. 여기서 생산능력이란 사업체가 설비,

# 6. streamlit 으로 간단하게 웹 ui 구현하기
1. [ngrok](https://dashboard.ngrok.com/) 에 접속 후 회원가입
2. 로그인 후 뜨는 authtoken 번호를 아래의 코드에 붙여넣기


In [16]:
!pip install streamlit
!pip install pyngrok

In [18]:
!streamlit hello & npx localtunnel --port 8501



⠙⠹⠸
  Welcome to Streamlit. Check out our demo in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.145.204:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y)   Stopping...
^C


In [90]:
%%writefile app.py

import streamlit as st
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import re

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정 (1000개로 확장)
)

#retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

#초기화
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어를 추천하세요.

단, 반드시 한국어로 답변해야 하며, 전체 Answer의 길이가 8줄을 넘어가지 않도록 하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

#gemini
PROJECT_ID = "your_project_id"
REGION = "your_region"

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

#streamlit 설계
st.set_page_config(page_title="금융용어알리미 : WhyFi")

st.markdown(
    """
    <h1 style='color: white; text-align: center;'>금융용어알리미 : WhyFi</h1>
    <h2 style='color: gray; text-align: center; font-size: 20px;'>모르는 금융 용어? 이제 쉽게 찾아보세요!</h2>
    <br>
    """,
    unsafe_allow_html=True,
)


user_question = st.text_input("금융 용어를 검색해보세요.", value="", placeholder="예: 가동률이란?")
if user_question:
    with st.spinner("답변을 생성 중입니다..."):
        result = chain({"question": user_question})

    # 결과 출력
    st.markdown("### 📖 답변 📖")

    # 텍스트에서 헤더(##)를 분리하고 스타일 적용
    answer_text = result["answer"]

    # 헤더와 본문 분리
    header_match = re.match(r"^##\s*(.*)", answer_text)
    if header_match:
        header = header_match.group(1)  # 헤더 부분
        body = re.sub(r"^##\s*.*\n*", "", answer_text, flags=re.MULTILINE)  # 나머지 텍스트
    else:
        header = ""
        body = answer_text

    # 헤더와 본문 각각 스타일링
    st.markdown(
        f"<div style='font-size:20px; font-weight:bold; line-height:1.8;'>{header}</div>",
        unsafe_allow_html=True,
    )
    st.markdown(
        f"<div style='font-size:16px; line-height:1.6;'>{body}</div>",
        unsafe_allow_html=True,
    )

    st.markdown("### 📚 참고 출처 📚")
    for doc in result["source_documents"]:
        st.markdown(
            f"<div style='font-size:14px; line-height:1.4;'>- {doc.metadata['source']}</div>",
            unsafe_allow_html=True,
        )

#         # 결과 출력
# st.markdown("#### 📖 답변:")
# st.markdown(f"<div style='font-size:16px; line-height:1.6;'>{result['answer']}</div>", unsafe_allow_html=True)

# st.markdown("#### 📚 참고 출처:")
# for doc in result["source_documents"]:
#     st.markdown(f"<div style='font-size:14px; line-height:1.4;'>- {doc.metadata['source']}</div>", unsafe_allow_html=True)


Overwriting app.py


In [91]:
!ls

app.py				      dataset					    RAG.ipynb
bf83c840-b72f-49be-95c1-4594b4c8eec1  gen-lang-client-0343826437-c90825b7f11b.json
chroma.sqlite3			      Langchain.ipynb


In [92]:
#개인 토큰 번호 입력
!ngrok authtoken [개인 토큰 번호]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [93]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&

In [81]:
publ_url = ngrok.connect(addr="8501")

In [82]:
publ_url

<NgrokTunnel: "https://2229-34-125-145-204.ngrok-free.app" -> "http://localhost:8501">

In [100]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:11 node
      9 ?        00:00:01 oom_monitor.sh
     11 ?        00:00:00 run.sh
     12 ?        00:00:01 kernel_manager_
     41 ?        00:00:00 tail
     47 ?        00:00:00 tail
     64 ?        00:00:12 python3 <defunct>
     65 ?        00:00:00 colab-fileshim.
    114 ?        00:00:06 jupyter-noteboo
    115 ?        00:00:03 dap_multiplexer
   2561 ?        00:00:00 bash
   2562 ?        00:00:00 drive
   2563 ?        00:00:00 grep
   2685 ?        00:00:03 drive
   2784 ?        00:00:00 bash
   2785 ?        00:00:00 directoryprefet
   2786 ?        00:00:00 tail
   2787 ?        00:00:00 python3
   3048 ?        00:00:52 python3
   3075 ?        00:00:07 python3
   5229 ?        00:00:02 language_servic
   5235 ?        00:11:21 node
  13023 ?        00:00:00 language_servic
  13029 ?        00:02:14 node
  17394 ?        00:00:00 sleep
  17395 ?        00:00:00 ps


In [97]:
!kill 13177 17180 12664

/bin/bash: line 1: kill: (13177) - No such process
/bin/bash: line 1: kill: (17180) - No such process
/bin/bash: line 1: kill: (12664) - No such process


In [99]:
ngrok.kill()